In [1]:
import numpy as np

# F_2
http://www.math.unl.edu/~s-jeverso2/McElieceProject.pdf <br>
$\mathbb{F}_{2}$

# F_23
https://www.youtube.com/watch?v=GNgOJN9LD-I <br>
https://jeremykun.com/2015/09/07/welch-berlekamp/ <br>
https://en.wikipedia.org/wiki/Berlekamp%E2%80%93Welch_algorithm <br>
$\mathbb{F}_{23}$

In [2]:
n = 23

In [3]:
m = np.array([1, 2, 3], dtype = int)

In [4]:
N = 6
K = 3
T = 1

In [5]:
G = np.array([
    [1, 1, 1, 1, 1, 1],
    [0, 1, 2, 3, 4, 5],
    [0, 1, 4, 9, 16, 25]], dtype = int) % n

P = np.array([
    [1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0]], dtype = int) % n

S = np.array([
    [1, 1, 3],
    [4, 1, 6],
    [7, 2, 9]], dtype = int) %n

In [59]:
for pi in SGP:
    for pii in pi[:-1]:
        print(pii,'&',end='')
    print(pi[-1],'\\\\')

1 &15 &7 &5 &12 &8 \\
4 &7 &12 &11 &21 &15 \\
7 &1 &21 &18 &12 &2 \\


In [6]:
SGP = S.dot(G).dot(P) % n
SGP

array([[ 1, 15,  7,  5, 12,  8],
       [ 4,  7, 12, 11, 21, 15],
       [ 7,  1, 21, 18, 12,  2]])

In [7]:
c = m.dot(SGP) % n
c

array([ 7,  9,  2, 12, 21, 21])

In [8]:
ei = np.random.randint(c.shape[0])
ev = np.random.randint(n)
err = np.zeros(c.shape[0], dtype = int)
err[ei] = ev
err = np.array([0, 6, 0, 0, 0, 0])
err

array([0, 6, 0, 0, 0, 0])

In [9]:
c = (c + err) % n
c

array([ 7, 15,  2, 12, 21, 21])

In [10]:
P1 = np.linalg.inv(P)
S1 = np.linalg.inv(S)

In [11]:
P1 

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.]])

In [22]:
c_m = c.dot(P1).astype(int)
c_m

array([ 7, 12, 15, 21,  2, 21])

### Berlekamp-Welch Algorithm

In [47]:
a = np.arange(N, dtype = int)
b = -c_m*a**2 
b = b.astype(int) % n
#b = np.concatenate([[b[0]], [1], b[1:]])

z = np.zeros(N)
z[1] = 1
BW = np.concatenate([[c_m], [c_m*a], [-np.ones(N)], [-a], [-a**2], [-a**3]], axis = 0).T
#BW = np.concatenate([[BW[0,:]], [z] ,BW[1:,:]], axis = 0) % n
BW = BW.astype(int) % n
BW

array([[ 7,  0, 22,  0,  0,  0],
       [12, 12, 22, 22, 22, 22],
       [15,  7, 22, 21, 19, 15],
       [21, 17, 22, 20, 14, 19],
       [ 2,  8, 22, 19,  7,  5],
       [21, 13, 22, 18, 21, 13]])

In [48]:
b

array([ 0, 11,  9, 18, 14,  4])

In [50]:
p = np.linalg.solve(BW,b)  % n
p

array([ 0.09032973,  0.13805661, 22.97125872,  0.97292092, 21.85273417,
        0.57851182])

In [51]:
E = np.poly1d([1, p[0]])  
Q = np.poly1d(p[1:])  

In [52]:
quotient, remainder = np.polydiv(Q, E) 

In [53]:
quotient

poly1d([ 0.13805661, 22.9587881 , -1.10094031, 21.95218182])

In [54]:
remainder

poly1d([-1.40442294])